In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")


# The weather API gives Max Temp in Kelvin. Converting to fahrenheit 
# formula: F = 1.8*(K-273) + 32.
city_data_df["Max Temp"] = (city_data_df["Max Temp"] -273.15)*1.8 +32

city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Henties Bay,NaN,2022-03-30 00:54:11,-22.1160,14.2845,62.564,86,1,2.41
1,1,Dunedin,NZ,2022-03-30 00:54:11,-45.8742,170.5036,66.794,71,0,4.92
2,2,Capinota,BO,2022-03-30 00:54:12,-17.7167,-66.2333,67.406,36,4,1.41
3,3,Trofors,NO,2022-03-30 00:54:12,65.5333,13.4063,15.170,97,87,1.54
4,4,Longyearbyen,SJ,2022-03-30 00:54:13,78.2186,15.6401,10.238,72,0,3.60


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

In [5]:
# Get the maximum temperature.
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

In [6]:
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of windiness
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Dunedin,NZ,2022-03-30 00:54:11,-45.8742,170.5036,66.794,71,0,4.92
2,2,Capinota,BO,2022-03-30 00:54:12,-17.7167,-66.2333,67.406,36,4,1.41
5,5,Carnarvon,AU,2022-03-30 00:54:13,-24.8667,113.6333,77.072,88,100,3.09
7,7,Pacific Grove,US,2022-03-30 00:54:14,36.6177,-121.9166,65.246,86,100,4.63
10,10,Mataura,NZ,2022-03-30 00:54:15,-46.1927,168.8643,72.482,50,4,0.07
...,...,...,...,...,...,...,...,...,...,...
538,538,Santa Isabel Do Rio Negro,BR,2022-03-30 01:09:53,-0.4139,-65.0192,72.896,99,99,0.31
539,539,Sai Buri,TH,2022-03-30 01:09:53,6.7013,101.6168,80.960,80,2,2.30
543,543,Nisia Floresta,BR,2022-03-30 01:09:54,-6.0911,-35.2086,77.810,89,100,3.13
544,544,Inhambane,MZ,2022-03-30 01:09:54,-23.8650,35.3833,72.716,72,31,2.55


In [12]:
preferred_cities_df.count()

City_ID       255
City          255
Country       254
Date          255
Lat           255
Lng           255
Max Temp      255
Humidity      255
Cloudiness    255
Wind Speed    255
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Dunedin,NZ,66.794,-45.8742,170.5036,
2,Capinota,BO,67.406,-17.7167,-66.2333,
5,Carnarvon,AU,77.072,-24.8667,113.6333,
7,Pacific Grove,US,65.246,36.6177,-121.9166,
10,Mataura,NZ,72.482,-46.1927,168.8643,
11,Richards Bay,ZA,72.068,-28.7830,32.0377,
14,Busselton,AU,69.980,-33.6500,115.3333,
18,Tambacounda,SN,88.898,13.7539,-13.7586,
21,Rikitea,PF,78.800,-23.1203,-134.9692,
28,Acapulco,MX,77.990,16.8634,-99.8901,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))